In [1]:
import numpy as np
import random

Things to try: 
self-unknown?
input cliques, instead of just pairs
distances between voters?
directed pairs of neighbors? e.g. celebrities
different local allocation rules (as keywords in allocate function)
different graphs, sparsities

In [2]:
# each voter is an index in the array

def construct_neighbors(truecompetencies, neighborpairs):
    """Returns direct neighbors of voters given pairs of neighbors (edges in graph)
    ARGS:
        truecompetencies: list of true competencies (between 0 and 1) for each voter
        neighborpairs: list of unrepeated, undirected pairs of neighbors; not necessarily ordered
    
    OUTPUTS:
        neighbors: list of direct neighbors for each voter
    """
    n = len(truecompetencies)
    
    if np.sum(np.array(neighborpairs).flatten() > n - 1) != 0:
        return("Error: Pair contains nonexistent voter.")
    
    # initialize neighborlist
    neighbors = [[] for i in range(n)]
    
    # add neighbors
    for pair in neighborpairs:
        (p1, p2) = pair
        neighbors[p1].append(p2)
        neighbors[p2].append(p1)

    return neighbors

In [3]:
# perceived competencies are just based on public voting record. extend to voting for representative?
# unless we want to consider if stop watching -- if i vote right i don't look at others?
def prior_competencies(competencies):
    """Initializes Beta(1,1) public competency distribution priors for everyone
    ARGS:
        competencies: list of true competencies (between 0 and 1) for each voter
    OUTPUTS:
        priors: list of Beta parameters for each individual
    """
    n = len(competencies)
    priors = [(1,1) for i in range(n)] # initialize everyone to Beta(1,1)=Unif(0,1) distributions
    return priors

In [4]:
def allocate(truecompetencies, pubcompetencies, neighbors, mechanism="standard"):
    """Returns index of final voter whom each voter allocates their vote to
    ARGS:
        truecompetencies: list of true competencies (between 0 and 1) for each voter
        pubcompetencies: list of public competency distributions for each voter, as paramters for Beta dist
        neighbors: list of direct neighbors for each voter
    Options:
        mechanism: local delegation mechanism to use (see below)
    Mechanisms:
        "standard": each voter delegates to their neighbor with highest mean competency.
                    in case of ties, voter delgates to the first in their list
        "truestandard": each voter delegates to their neighbor with highest true competency.
                    in case of ties, voter delgates to the first in their list
                    
        TODO: other (non-deterministic) mechanisms?
    
    OUTPUTS:
        allocation: list of indices, where each value is the index of the sink
                    voter whom that voter delegates to
    """
    
    allocation = [i for i in range(len(truecompetencies))] # initially each voter allocates vote to self
    
    meanpubcompetencies = [j[0]/(j[0]+j[1]) for j in pubcompetencies] # mean of each dist
    
    # for every voter, consider their neighbors
    for i, ineighbors in enumerate(neighbors):
        # i is index of voter being considered, ineighbors is list of indices of i's neighbors
        
        # if i has no neighbors
        if len(ineighbors) == 0:
            continue
        
        elif mechanism=="standard":
            neighborcompetencies = [meanpubcompetencies[j] for j in ineighbors] 
            bestneighbor = ineighbors[np.argmax(neighborcompetencies)] # picks neighbor with highest competency; if ties, chooses first
            
            if truecompetencies[i] < meanpubcompetencies[bestneighbor]:
                allocation[i] = bestneighbor
                
        elif mechanism=="truestandard":
            neighborcompetencies = [truecompetencies[j] for j in ineighbors]
            bestneighbor = ineighbors[np.argmax(neighborcompetencies)] # picks neighbor with highest competency; if ties, chooses first

            if truecompetencies[i] < truecompetencies[bestneighbor]:
                allocation[i] = bestneighbor
        
        # TODO: other mechanisms
        
            
    # Here we allocate to the last sink in the chain
    for node in range(len(allocation)): # check across all voters
        path = [node] # create running cycle list 
        
        while allocation[path[-1]] not in path: # if hasn't gotten to someone who we've seen before
            # print(path)
            path.append(allocation[path[-1]]) # add them to the running list of voters in cycle
            
        delegate = random.choice(path[path.index(allocation[path[-1]]):]) # pick someone random in the cycle (cycle starts from point where while loop broken)
        
        for point_to_sink in path: # make everyone delegate to same person, completing liquid flow
            allocation[point_to_sink] = delegate
                
    return allocation

In [6]:
star_truecomps = np.array([.8]+[.6 for i in range(8)]) # true competencies
star_pubcomps = prior_competencies(star_truecomps) # initialize parameters for priors
star_nbpairs = np.array([(0, i) for i in range(1,9)]) # pairs of neighbors
print(star_truecomps, star_pubcomps, star_nbpairs)

[0.8 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6] [(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)] [[0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]
 [0 7]
 [0 8]]


In [7]:
# test once
star_nbs = construct_neighbors(star_truecomps, star_nbpairs)
print(allocate(star_truecomps, star_pubcomps, star_nbs, mechanism="standard")) # allocate using mean perceived competency
print(allocate(star_truecomps, star_pubcomps, star_nbs, mechanism="truestandard")) # allocate using true competency

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 0, 0, 0, 0, 0, 0, 0, 0]


In [5]:
def update_competencies(initialcompetencies, votes):
    """Returns updated competencies
    ARGS:
        initialcompetencies: list of initial public competency distributions for each voter, as parameters for Beta dist
        votes: list of voting correctness for each voter
        
    OUTPUTS:
        finalcompetencies: list of initial public competency distributions for each voter, as parameters for Beta dist
    """
    finalcompetencies = initialcompetencies.copy()
    for i in range(len(votes)):
        finalcompetencies[i] = (initialcompetencies[i][0] + 1, initialcompetencies[i][1]) if votes[i]==1 else (initialcompetencies[i][0], initialcompetencies[i][1] + 1)
    
    return finalcompetencies

In [9]:
star_pubcomps
testvotes = [1 for i in range(5)] + [0 for i in range(4)]
star_pubcomps2 = update_competencies(star_pubcomps, testvotes)
star_pubcomps2

[(2, 1), (2, 1), (2, 1), (2, 1), (2, 1), (1, 2), (1, 2), (1, 2), (1, 2)]

In [6]:
def simulate_liquid(truecompetencies, neighborpairs, pubcompetencies=None, rounds=100, mechanism="standard"):
    """Returns final public competencies, arrays of numbers = np.random.binomial(n, p, 1000)s = np.random.binomial(n, p, 1000) of voters who allocate, nubmer of voters who vote correctly
    ARGS:
        truecompetencies: list of true competencies (between 0 and 1) for each voter
        neighborpairs: list of unrepeated, undirected pairs of neighbors; not necessarily ordered
    Options:
        pubcompetencies: list of public competency distributions for each voter, as parameters for Beta dist
        rounds: # of iterative rounds to vote
        mechanism: local delegation mechanism to use (see allocate function)
        
    OUTPUTS:
        pubcompetencies: list of final public competency distributions for each voter, as parameters for Beta dist
        numallocated: number of voters who allocate their votes in each round
        numcorrect: number of voters who vote correctly in each round
        numnoallocate: control test TODO: delete
    """
    
    n = len(truecompetencies)
    neighbors = construct_neighbors(truecompetencies, neighborpairs)
    
    if pubcompetencies==None:
        pubcompetencies = prior_competencies(truecompetencies) # initialize to Beta(1,1)
    
    numallocated = []
    numcorrect = []
    numnoallocate = [] # can ignore: testing, for comparison
    for t in range(rounds):
        # allocate
        allocation = allocate(truecompetencies, pubcompetencies, neighbors, mechanism="standard")
        # vote
        initialvotes = np.random.binomial(1, truecompetencies)
        allocatedvotes = initialvotes[allocation]
        
        allocated = np.count_nonzero(np.array(allocation)-np.array([i for i in range(len(truecompetencies))])) # number of voters who allocated their vote to someone else
        
        numallocated.append(allocated) # add number of voters who give up their vote
        numcorrect.append(np.sum(allocatedvotes)) # add number correct to correct
        numnoallocate.append(np.sum(initialvotes)) # add initial number correct to noallocate

        # update perceived competencies
        pubcompetencies = update_competencies(pubcompetencies, initialvotes) # NOTE: judging everyone based on their broadcasted vote (possibly not counted)
        # pubcompetencies = update_competencies(pubcompetencies, allocatedvotes) #NOTE: judging everyone by their final (possibly allocated) vote!!!!
    
    propcorrect = np.sum(numcorrect>np.ceil((n+1)/2.))/rounds # proportion over time of votes that are "correct" (>1/2 of voters vote correctly)
    propnoallocate = np.sum(numnoallocate>np.ceil((n+1)/2.))/rounds
    print("Proportion correct:", propcorrect, " Proportion noallocate right: ", propnoallocate)
    return pubcompetencies, np.array(numallocated), np.array(numcorrect), np.array(numnoallocate)

In [11]:
# Star structure 1
for center_competency in [0.0, 0.2, 0.4, 0.6, 0.7, 0.8, 0.9, 1.0]:
    star_truecomps = np.array([center_competency]+[.6 for i in range(8)]) # true competencies
    star_nbpairs = np.array([(0, i) for i in range(1,9)]) # pairs of neighbors
    print("Center competency:", center_competency)
    simulate_liquid(star_truecomps, star_nbpairs, rounds=1000, mechanism="standard")
    # simulate_liquid(star_truecomps, star_nbpairs, rounds=100, mechanism="truestandard")

Center competency: 0.0
Proportion correct: 0.482  Proportion noallocate right:  0.301
Center competency: 0.2
Proportion correct: 0.535  Proportion noallocate right:  0.403
Center competency: 0.4
Proportion correct: 0.465  Proportion noallocate right:  0.397
Center competency: 0.6
Proportion correct: 0.49  Proportion noallocate right:  0.474
Center competency: 0.7
Proportion correct: 0.689  Proportion noallocate right:  0.513
Center competency: 0.8
Proportion correct: 0.807  Proportion noallocate right:  0.539
Center competency: 0.9
Proportion correct: 0.899  Proportion noallocate right:  0.556
Center competency: 1.0
Proportion correct: 0.999  Proportion noallocate right:  0.601


In [12]:
# Star structure 2
for center_competency in [0.0, 0.2, 0.4, 0.6, 0.7, 0.8, 0.9, 1.0]:
    star_truecomps = np.array([center_competency]+[.6 for i in range(8)]) # true competencies
    star_nbpairs = np.array([(0, i) for i in range(1,9)]) # pairs of neighbors
    print("Center competency:", center_competency)
    simulate_liquid(star_truecomps, star_nbpairs, rounds=1000, mechanism="standard")
    # simulate_liquid(star_truecomps, star_nbpairs, rounds=100, mechanism="truestandard")

Center competency: 0.0
Proportion correct: 0.476  Proportion noallocate right:  0.303
Center competency: 0.2
Proportion correct: 0.498  Proportion noallocate right:  0.382
Center competency: 0.4
Proportion correct: 0.498  Proportion noallocate right:  0.446
Center competency: 0.6
Proportion correct: 0.508  Proportion noallocate right:  0.475
Center competency: 0.7
Proportion correct: 0.691  Proportion noallocate right:  0.521
Center competency: 0.8
Proportion correct: 0.766  Proportion noallocate right:  0.536
Center competency: 0.9
Proportion correct: 0.903  Proportion noallocate right:  0.582
Center competency: 1.0
Proportion correct: 0.999  Proportion noallocate right:  0.599


In [7]:
# Generate popularities (not being used atm)

def generate_popularities(n, alpha = 1, x_m = 1):
    """Returns voters' popularities
    ARGS:
        n: number of voters
        
    OUTPUTS:
        popularities: list of popularity for each voter, drawn from a Pareto distribution
    """
    
    popularities = (np.random.pareto(alpha, n) + 1) * x_m # number of people each person knows
    
    for i in range(n):
        popularities[i] = min(1, popularities[i] / n)

    print("Popularities:", popularities)
    
#     Plot PDF

#     import matplotlib.pyplot as plt
#     count, bins, _ = plt.hist(popularities, 100, density=True)
#     fit = alpha*x_m**alpha / bins**(alpha+1)
#     plt.plot(bins, max(count)*fit/max(fit), linewidth=2, color='r')
#     plt.show()
    

In [14]:
# Construct neighborpairs for a random graph

def construct_neighborpairs(n, p = 0.01):
    """Returns neighbor pairs
    ARGS:
        n: number of voters
        p: probability that i and j are connected
        
    OUTPUTS:
        neighborpairs: list of pairs of neighbors
    """
    
    neighborpairs = []
    
    for i in range(n):
        for j in range(i+1, n):
            connected = np.random.binomial(1, p)
            if connected:
                neighborpairs.append((i, j))
    return neighborpairs

In [15]:
# Random graph simulation

n = 100 # number of people
rand_truecomps = np.random.random_sample(size = n)
rand_nbpairs = construct_neighborpairs(n)
simulate_liquid(rand_truecomps, rand_nbpairs, rounds=50, mechanism="standard")


Proportion correct: 0.96  Proportion noallocate right:  0.26


([(41, 11),
  (9, 43),
  (10, 42),
  (12, 40),
  (24, 28),
  (29, 23),
  (32, 20),
  (50, 2),
  (19, 33),
  (42, 10),
  (3, 49),
  (11, 41),
  (2, 50),
  (30, 22),
  (26, 26),
  (51, 1),
  (1, 51),
  (51, 1),
  (38, 14),
  (43, 9),
  (46, 6),
  (46, 6),
  (44, 8),
  (16, 36),
  (9, 43),
  (24, 28),
  (35, 17),
  (38, 14),
  (27, 25),
  (8, 44),
  (35, 17),
  (29, 23),
  (27, 25),
  (8, 44),
  (9, 43),
  (18, 34),
  (44, 8),
  (38, 14),
  (1, 51),
  (46, 6),
  (29, 23),
  (51, 1),
  (11, 41),
  (21, 31),
  (37, 15),
  (5, 47),
  (9, 43),
  (39, 13),
  (11, 41),
  (19, 33),
  (5, 47),
  (6, 46),
  (20, 32),
  (8, 44),
  (39, 13),
  (28, 24),
  (43, 9),
  (43, 9),
  (16, 36),
  (16, 36),
  (5, 47),
  (39, 13),
  (22, 30),
  (9, 43),
  (27, 25),
  (30, 22),
  (15, 37),
  (35, 17),
  (6, 46),
  (20, 32),
  (48, 4),
  (2, 50),
  (2, 50),
  (43, 9),
  (30, 22),
  (38, 14),
  (4, 48),
  (45, 7),
  (7, 45),
  (49, 3),
  (30, 22),
  (28, 24),
  (45, 7),
  (2, 50),
  (8, 44),
  (38, 14),
  (39, 1

In [16]:
# Simulate a bunch, 50 rounds

for i in range(100):
    n = 100 # number of people
    rand_truecomps = np.random.random_sample(size = n)
    rand_nbpairs = construct_neighborpairs(n)
    simulate_liquid(rand_truecomps, rand_nbpairs, rounds=50, mechanism="standard")
    
    print(i+1, "th iteration complete!!-------------------------------------------------------")

Proportion correct: 1.0  Proportion noallocate right:  0.54
1 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.94  Proportion noallocate right:  0.24
2 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.96  Proportion noallocate right:  0.32
3 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.98  Proportion noallocate right:  0.14
4 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.98  Proportion noallocate right:  0.08
5 th iteration complete!!-------------------------------------------------------
Proportion correct: 1.0  Proportion noallocate right:  0.68
6 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.96  Proportion noallocate right:  0.16
7 th iteration complete!!-------------------------------------------------------
Proporti

In [26]:
# Simulate a bunch, 5 rounds

for i in range(100):
    n = 10 # number of people
    rand_truecomps = np.random.random_sample(size = n)
    rand_nbpairs = construct_neighborpairs(n)
    simulate_liquid(rand_truecomps, rand_nbpairs, rounds=1, mechanism="standard")
    
    print(i+1, "th iteration complete!!-------------------------------------------------------")

Proportion correct: 0.0  Proportion noallocate right:  0.0
1 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.0  Proportion noallocate right:  0.0
2 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.0  Proportion noallocate right:  0.0
3 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.0  Proportion noallocate right:  0.0
4 th iteration complete!!-------------------------------------------------------
Proportion correct: 1.0  Proportion noallocate right:  1.0
5 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.0  Proportion noallocate right:  0.0
6 th iteration complete!!-------------------------------------------------------
Proportion correct: 0.0  Proportion noallocate right:  0.0
7 th iteration complete!!-------------------------------------------------------
Proportion correct: 

# Pareto Playground - Benji

In [17]:
import scipy.stats
scipy.stats.pareto.pdf(1,1) # = 1. normalized correctly
scipy.stats.pareto.pdf(5,1) # = .04
100*scipy.stats.pareto.pdf(1,1/100) # = 1. normalized correctly
100*scipy.stats.pareto.pdf(5,1/100) # = .2

# apply zipf?

0.19680688867269153

In [48]:
# Generate popularities according to Pareto dist

def generate_popularities(n, repeat=1):
    """Returns voters' popularities according to Zipf dist
    ARGS:
        n: number of voters
        repeat: number of voters with 1/i popularity
        
    OUTPUTS:
        popularities: list of popularity for each voter, following an approximate Zipf distribution
    """
    
    popularities = [1/np.ceil(i/repeat) for i in range(1,n+1)] # repeated Zipf function
    return popularities

    print("Popularities:", popularities)


def generate_zipfgraph(n, repeat=1):
    """Returns voters' popularities according to Zipf dist
    ARGS:
        n: number of voters
        repeat: number of voters with 1/i popularity
        
    OUTPUTS:
        popularities: list of popularity for each voter, following an approximate Zipf distribution
        nbpairs: generated pairs of neighbors, based on popularity
    """
    
    popularities = generate_popularities(n, repeat)
    
    nbpairs = []
    for i in range(n):
        for j in range(i+1,n):
            if random.random() < popularities[i]*popularities[j]:
                nbpairs.append((i,j))
    print(len(nbpairs), nbpairs)
    
    return popularities, nbpairs

generate_zipfgraph(100, repeat=3)

73 [(0, 1), (0, 2), (0, 3), (0, 4), (0, 6), (0, 7), (0, 31), (0, 38), (0, 42), (0, 58), (0, 64), (0, 69), (0, 78), (0, 94), (1, 2), (1, 3), (1, 6), (1, 8), (1, 17), (1, 21), (1, 23), (1, 38), (1, 47), (2, 5), (2, 6), (2, 7), (2, 16), (2, 21), (2, 78), (2, 81), (2, 94), (2, 96), (3, 17), (3, 49), (3, 98), (4, 5), (4, 78), (4, 89), (5, 11), (5, 36), (5, 50), (6, 13), (6, 26), (7, 16), (7, 37), (7, 95), (8, 14), (8, 32), (9, 36), (9, 37), (9, 38), (10, 40), (11, 68), (12, 16), (13, 34), (14, 26), (14, 39), (15, 39), (15, 46), (16, 18), (19, 70), (20, 46), (21, 34), (21, 40), (28, 29), (29, 70), (33, 40), (33, 50), (34, 41), (35, 89), (37, 92), (54, 58), (65, 95)]


([1.0,
  1.0,
  1.0,
  0.5,
  0.5,
  0.5,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.25,
  0.25,
  0.25,
  0.2,
  0.2,
  0.2,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.125,
  0.125,
  0.125,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1,
  0.1,
  0.1,
  0.09090909090909091,
  0.09090909090909091,
  0.09090909090909091,
  0.08333333333333333,
  0.08333333333333333,
  0.08333333333333333,
  0.07692307692307693,
  0.07692307692307693,
  0.07692307692307693,
  0.07142857142857142,
  0.07142857142857142,
  0.07142857142857142,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.0625,
  0.0625,
  0.0625,
  0.058823529411764705,
  0.058823529411764705,
  0.058823529411764705,
  0.05555555555555555,
  0.05555555555555555,
  0.05555555555555555,
  0.05263157894736842,
  0.05263157894736842,
  0.05263157894736842,
  0.05,
  

In [49]:
for i in range(10):
    n = 100 # number of people
    zipf_truecomps, zipf_nbpairs = generate_zipfgraph(n, repeat=3)
    simulate_liquid(zipf_truecomps, zipf_nbpairs, rounds=100, mechanism="standard")
    
    print(i+1, "th iteration complete!!-------------------------------------------------------")

57 [(0, 1), (0, 2), (0, 3), (0, 4), (0, 6), (0, 16), (0, 36), (0, 62), (0, 76), (0, 79), (1, 2), (1, 3), (1, 5), (1, 8), (1, 13), (1, 16), (1, 18), (1, 19), (1, 45), (1, 72), (1, 87), (1, 90), (2, 4), (2, 5), (2, 7), (2, 8), (2, 9), (2, 18), (2, 37), (2, 75), (2, 77), (3, 14), (3, 57), (3, 68), (4, 9), (4, 33), (4, 38), (4, 41), (5, 7), (5, 8), (5, 16), (5, 72), (5, 80), (6, 62), (7, 20), (8, 71), (9, 16), (11, 59), (13, 60), (14, 71), (15, 26), (22, 97), (23, 46), (26, 42), (31, 63), (32, 83), (39, 62)]
Proportion correct: 0.0  Proportion noallocate right:  0.0
1 th iteration complete!!-------------------------------------------------------
92 [(0, 1), (0, 2), (0, 3), (0, 5), (0, 8), (0, 17), (0, 19), (0, 41), (0, 48), (0, 66), (0, 75), (1, 2), (1, 4), (1, 9), (1, 10), (1, 15), (1, 18), (1, 28), (1, 29), (1, 31), (1, 32), (1, 77), (2, 3), (2, 4), (2, 12), (2, 21), (2, 26), (2, 27), (2, 47), (2, 57), (2, 58), (3, 5), (3, 6), (3, 11), (3, 15), (3, 16), (3, 18), (3, 28), (4, 17), (4, 44)

In [ ]:
# TODO: grid structure